In [3]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

In [4]:
#Actions: hit and stick
actionHit = 0
actionStick = 1
actions = [actionHit, actionStick]

In [14]:
#policy for the player
policyPlayer = np.zeros(22)
for i in range(12, 20):
    policyPlayer[i] = actionHit
policyPlayer[20] = actionStick
policyPlayer[21] = actionStick

In [7]:
#function form of target policy of the player
def targetPolicyPlayer(usableAcePlayer, playerSum, dealerCard):
    return policyPlayer[playerSum]

In [13]:
#function form of behaviour policy of the player
def behaviourPolicyPlayer(usableAcePlayer, playerSum, dealerCard):
    if np.random.binomial(1, 0.5) == actionStick:
        return actionStick
    return actionHit

In [15]:
#policy for dealer
policyDealer = np.zeros(22)
for i in range(12, 17):
    policyDealer[i] = actionHit
for i in range(17, 22):
    policyDealer[i] = actionStick

In [17]:
#function to get new card
def getCard():
    card = np.random.choice(1, 14)
    card = min(card, 10) #face cards have value equal to 10
    return card

#function to get value of a card
def cardValue(cardid):
    return 11 if cardid == 1 else cardid

In [ ]:
def play(policyPlayer, initialState = None, initialAction = None):
    #player Status 
    #Sum of player cards
    playerSum = 0
    
    #Trajectory of player
    playerTrajectory = []
    
    #Usable ace or not
    useableAcePlayer = False
    
    #Dealer Status
    dealerCard1 = 0
    dealerCard2 = 0
    useableAceDealer = False
    
    if initialState is None:
        #Generate random initial State
        
        while playerSum < 12:
            card = getCard()
            playerSum += cardValue(card)
            
            if playerSum > 21:
                assert playerSum == 22 #Last card must be ace
                playerSum -= 10
            else:
                useableAcePlayer |= (1 == card)
        
        dealerCard1 = getCard()
        dealerCard2 = getCard()
        
    else:
        #Use specified initial state
        useableAcePlayer, playerSum, dealerCard1 = initialState
        dealerCard2 = getCard()
        
    state = [useableAcePlayer, playerSum, dealerCard1]
    
    dealerSum = cardValue(dealerCard1) + cardValue(dealerCard2)
    useableAceDealer = 1 in (dealerCard1, dealerCard2)
    
    #if the sum is greater than 21, dealer must hold 2 aces
    if dealerSum > 21:
        assert dealerSum == 22
        dealerSum -= 10 # Use one ace 1 rather than 11
    assert dealerSum <= 21
    assert playerSum <= 21
    
    #Game starts
    
    #Player's Turn
    while True:
        if initialAction is not None:
            action = initialAction
            initialAction = None
        else:
            #get action based on current turn
            action = policyPlayer(useableAcePlayer, playerSum, dealerCard1)
            
        #track player's tracjectory for importance sampling 
        playerTrajectory.append([(useableAcePlayer, playerSum, dealerCard1), action])
        
        if action == actionStick:
            break
    